In [1]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, max, min, count, broadcast, desc, asc, when, lit,row_number, rank, round
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, FloatType

In [2]:
def control_null (dataframe): 
    for dfcol in dataframe.columns:
        df_null = dataframe.filter(col(dfcol).isNull()).count()
        if df_null > 0:
            print(f"{dfcol} : {df_null} null")
        else:
            print(f"{dfcol} no tiene null")

In [3]:
spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .getOrCreate()
)

spark.sparkContext.setLogLevel("INFO")

logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)

logger.info("Log de ejemplo guardado en archivo y consola.")

errores_detectados = []


your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


14:02:34.098 [Thread-3] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [4]:
try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))
    
try:
    read_parquet_airline = config["Parquet_file"]["df_airline"]
    read_parquet_flights = config["Parquet_file"]["df_flights"]
    read_parquet_airports = config["Parquet_file"]["df_airports"]
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))


In [5]:
try:
    df_flights = spark.read.parquet(read_parquet_flights)
    df_airline = spark.read.parquet(read_parquet_airline)
    df_airports = spark.read.parquet(read_parquet_airports)
except Exception as error:
    logger.error("Error read.parquet:" + str(error))




flight_row_count = df_flights.count()

airline_row_count = df_airline.count()

airports_row_count = df_airports.count()

14:02:34.253 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
14:02:34.263 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Warehouse path is 'file:/home/illidan/proyecto_desde0/ETL/spark-warehouse'.
14:02:34.280 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@11ca1834{/SQL,null,AVAILABLE,@Spark}
14:02:34.283 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@2c10dc7e{/SQL/json,null,AVAILABLE,@Spark}
14:02:34.286 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@47b8caef{/SQL/execution,null,AVAILABLE,@Spark}
14:02:34.288 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@2b3a2190{/SQL/execution/json,null,AVAILABLE,@Spark}
14:02:34.304 [Threa

14:02:37.219 [Executor task launch worker for task 0.0 in stage 0.0 (TID 0)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 0.0 (TID 0). 3053 bytes result sent to driver
14:02:37.234 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 0.0 (TID 0) in 582 ms on 172.23.57.81 (executor driver) (1/1)
14:02:37.236 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 0.0, whose tasks have all completed, from pool 
14:02:37.242 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ResultStage 0 (parquet at NativeMethodAccessorImpl.java:0) finished in 0.871 s
14:02:37.247 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
14:02:37.247 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Killing all running tasks in stage 0: Stage finish

14:02:37.531 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_0_piece0 on 172.23.57.81:35957 in memory (size: 36.9 KiB, free: 434.4 MiB)
14:02:38.182 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.InMemoryFileIndex - It took 6 ms to list leaf files for 1 paths.
14:02:38.255 [Thread-3] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
14:02:38.257 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 1 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
14:02:38.257 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 1 (parquet at NativeMethodAccessorImpl.java:0)
14:02:38.257 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List()
14:02:38.257 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: Lis

14:02:40.512 [Executor task launch worker for task 5.0 in stage 3.0 (TID 8)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 3.0 (TID 8). 2222 bytes result sent to driver
14:02:40.518 [Executor task launch worker for task 0.0 in stage 3.0 (TID 3)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 3.0 (TID 3). 2222 bytes result sent to driver
14:02:40.518 [Executor task launch worker for task 2.0 in stage 3.0 (TID 5)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 3.0 (TID 5). 2222 bytes result sent to driver
14:02:40.519 [Executor task launch worker for task 3.0 in stage 3.0 (TID 6)] INFO  org.apache.spark.executor.Executor - Finished task 3.0 in stage 3.0 (TID 6). 2222 bytes result sent to driver
14:02:40.523 [Executor task launch worker for task 7.0 in stage 3.0 (TID 10)] INFO  org.apache.spark.executor.Executor - Finished task 7.0 in stage 3.0 (TID 10). 2222 bytes result sent to driver
14:02:40.524 [task-result-getter-

14:02:40.966 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: 
14:02:40.966 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: 
14:02:41.000 [Thread-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_6 stored as values in memory (estimated size 200.1 KiB, free 434.0 MiB)
14:02:41.026 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_5_piece0 on 172.23.57.81:35957 in memory (size: 5.9 KiB, free: 434.4 MiB)
14:02:41.030 [Thread-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_6_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.9 MiB)
14:02:41.031 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Added broadcast_6_piece0 in memory on 172.23.57.81:35957 (size: 34.5 KiB, free: 434.3 MiB)
14:02:41.033 [Thread-3] INFO  org.apache.spark.SparkContext - Created

In [6]:
try:
    #avg per fly
    avg_flight = df_flights.join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                        .groupBy(df_flights.AIRLINE, df_airline.AIRLINE) \
                        .agg(round(avg(df_flights.DISTANCE), 2).alias("avg_DISTANCE")) \
                        .orderBy(desc("avg_DISTANCE")) \
                        .select(df_airline.AIRLINE.alias("AIRLINE_Name"), "avg_DISTANCE")
except Exception as error:
    logger.error("Error variable avg_flight:" + str(error))


14:02:41.618 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_6_piece0 on 172.23.57.81:35957 in memory (size: 34.5 KiB, free: 434.3 MiB)


In [7]:
try:
    #how many times a flight visit an airport
    airline_in_airport = df_flights.join(broadcast(df_airports), df_flights.ORIGIN_AIRPORT == df_airports.IATA_CODE) \
                                .join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                                .repartition(df_flights.DESTINATION_AIRPORT) \
                                .groupBy(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE) \
                                    .agg(count(df_flights.AIRLINE).alias("Count_visit_per_airline")) \
                                .orderBy(asc(df_flights.DESTINATION_AIRPORT)) \
                                .select(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE,"Count_visit_per_airline")
except Exception as error:
    logger.error("Error variable airline_in_airport:" + str(error))

#a Rank of wich airline is the most visited in each airport
try:
    window_spec = Window.partitionBy("DESTINATION_AIRPORT").orderBy(desc("Count_visit_per_airline"))
    
    Rank_airline_in_airport = airline_in_airport.withColumn("Ranking", rank().over(window_spec))


except Exception as error:
    logger.error("Error Rank_airline_in_airport:" + str(error))


In [8]:
try:
    #mention how many times an airline visit a destination and canceled this arrival
    flights_per_cancell = df_flights.select(col("AIRLINE"), col("CANCELLED"), col("DESTINATION_AIRPORT")) \
                                .filter(col("CANCELLED") == "1") \
                                .repartition(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                .groupBy(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                    .agg(count(col("AIRLINE")).alias("count_airlines_cancel")) \
                                .orderBy(desc("DESTINATION_AIRPORT")) \
                                .limit(100)
except Exception as error:
    logger.error("Error variable flights_per_cancell:" + str(error))


In [9]:
try:
    #i create this variable because i need to know how many airports have flights but are not in the list of airports
    airport_notin_thelist = df_flights.join(broadcast(df_airports), df_flights.DESTINATION_AIRPORT == df_airports.IATA_CODE, "left_anti") \
                                        .select("DESTINATION_AIRPORT") \
                                        .repartition("DESTINATION_AIRPORT") \
                                        .groupBy("DESTINATION_AIRPORT").agg(count("*").alias("Count_airports"))
except Exception as error:
    logger.error("Error variable airport_notin_thelist:" + str(error))


In [10]:
#Create parquet files

try:
    avg_flight.write.parquet(config["Parquet_file"]["avg_flight"], mode="overwrite")

    Rank_airline_in_airport.write.parquet(config["Parquet_file"]["airline_in_airport"], mode="overwrite")

    flights_per_cancell.write.parquet(config["Parquet_file"]["flights_per_cancell"], mode="overwrite")

    airport_notin_thelist.write.parquet(config["Parquet_file"]["airport_notin_thelist"], mode="overwrite")
except Exception as error:
    logger.error("Error write parquet files:" + str(error))


14:02:42.704 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(AIRLINE)
14:02:42.704 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(AIRLINE#4)
14:02:42.709 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(IATA_CODE)
14:02:42.709 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(IATA_CODE#62)
14:02:42.764 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_11_piece0 on 172.23.57.81:35957 in memory (size: 5.9 KiB, free: 434.3 MiB)
14:02:42.772 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_9_piece0 on 172.23.57.81:35957 in memory (size: 34.5 KiB, free: 434.4 MiB)
14:02:42.851 [broadcast-exchange-0] INFO  org.apache.spark.sql.catalyst.expressions.codegen.C

14:02:45.893 [Executor task launch worker for task 1.0 in stage 13.0 (TID 18)] INFO  org.apache.spark.executor.Executor - Finished task 1.0 in stage 13.0 (TID 18). 3926 bytes result sent to driver
14:02:45.897 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 1.0 in stage 13.0 (TID 18) in 2091 ms on 172.23.57.81 (executor driver) (1/8)
14:02:45.900 [Executor task launch worker for task 0.0 in stage 13.0 (TID 17)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 13.0 (TID 17). 3926 bytes result sent to driver
14:02:45.905 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 13.0 (TID 17) in 2098 ms on 172.23.57.81 (executor driver) (2/8)
14:02:45.908 [Executor task launch worker for task 2.0 in stage 13.0 (TID 19)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 13.0 (TID 19). 3969 bytes result sent to driver
14:02:45.914 [task-result-getter-2] INFO  org.apache.

14:02:46.556 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 1
14:02:46.557 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
14:02:46.559 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol - Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
14:02:46.559 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 1
14:02:46.559 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
14:02:46.561 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol - Using output 

14:02:48.716 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_14_piece0 on 172.23.57.81:35957 in memory (size: 686.0 B, free: 434.2 MiB)
14:02:48.743 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_12_piece0 on 172.23.57.81:35957 in memory (size: 34.6 KiB, free: 434.3 MiB)
14:02:48.787 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_15_piece0 on 172.23.57.81:35957 in memory (size: 34.7 KiB, free: 434.3 MiB)
14:02:49.408 [Executor task launch worker for task 6.0 in stage 23.0 (TID 36)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 23.0 (TID 36). 2451 bytes result sent to driver
14:02:49.414 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 23.0 (TID 36) in 1529 ms on 172.23.57.81 (executor driver) (1/8)


14:02:50.478 [Executor task launch worker for task 5.0 in stage 23.0 (TID 35)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 23.0 (TID 35). 2494 bytes result sent to driver
14:02:50.489 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 5.0 in stage 23.0 (TID 35) in 2604 ms on 172.23.57.81 (executor driver) (2/8)
14:02:50.535 [Executor task launch worker for task 1.0 in stage 23.0 (TID 31)] INFO  org.apache.spark.executor.Executor - Finished task 1.0 in stage 23.0 (TID 31). 2451 bytes result sent to driver
14:02:50.537 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 1.0 in stage 23.0 (TID 31) in 2654 ms on 172.23.57.81 (executor driver) (3/8)
14:02:50.579 [Executor task launch worker for task 0.0 in stage 23.0 (TID 30)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 23.0 (TID 30). 2451 bytes result sent to driver
14:02:50.581 [task-result-getter-1] INFO  org.apache.

...

14:02:54.185 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_30_piece0 on 172.23.57.81:35957 in memory (size: 95.3 KiB, free: 434.2 MiB)
14:02:54.196 [Executor task launch worker for task 0.0 in stage 31.0 (TID 59)] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 29.885705 ms
14:02:54.199 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_29_piece0 on 172.23.57.81:35957 in memory (size: 27.5 KiB, free: 434.2 MiB)
14:02:54.202 [Executor task launch worker for task 6.0 in stage 31.0 (TID 65)] INFO  org.apache.spark.sql.execution.datasources.FileScanRDD - Reading File path: file:///home/illidan/proyecto_desde0/archivos_parquet/df_flights/part-00006-b34af671-d5d7-439b-bf6c-1fc0fdc158b1-c000.snappy.parquet, range: 0-17101541, partition values: [empty row]
14:02:54.203 [Executor task launch worker for task 0.0 in stage 31.0 (TID 59)] INFO  org.apache.

14:02:54.732 [Executor task launch worker for task 3.0 in stage 31.0 (TID 62)] INFO  org.apache.spark.executor.Executor - Finished task 3.0 in stage 31.0 (TID 62). 2339 bytes result sent to driver
14:02:54.745 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 3.0 in stage 31.0 (TID 62) in 591 ms on 172.23.57.81 (executor driver) (1/8)
14:02:54.815 [Executor task launch worker for task 7.0 in stage 31.0 (TID 66)] INFO  org.apache.spark.executor.Executor - Finished task 7.0 in stage 31.0 (TID 66). 2296 bytes result sent to driver
14:02:54.820 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 7.0 in stage 31.0 (TID 66) in 661 ms on 172.23.57.81 (executor driver) (2/8)


14:02:54.940 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_28_piece0 on 172.23.57.81:35957 in memory (size: 26.0 KiB, free: 434.2 MiB)
14:02:54.966 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_24_piece0 on 172.23.57.81:35957 in memory (size: 686.0 B, free: 434.2 MiB)
14:02:54.976 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_27_piece0 on 172.23.57.81:35957 in memory (size: 9.0 KiB, free: 434.3 MiB)
14:02:54.986 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_26_piece0 on 172.23.57.81:35957 in memory (size: 34.8 KiB, free: 434.3 MiB)
14:02:54.997 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_20_piece0 on 172.23.57.81:35957 in memory (size: 34.6 KiB, free: 434.3 MiB)
14:02:55.007 [dispatcher-BlockManagerMaster] INFO  org.ap

14:02:55.688 [Executor task launch worker for task 0.0 in stage 34.0 (TID 68)] INFO  org.apache.spark.sql.execution.datasources.FileScanRDD - Reading File path: file:///home/illidan/proyecto_desde0/archivos_parquet/df_airports/part-00000-10922ef7-e1bf-447c-b966-c26b9b507f33-c000.snappy.parquet, range: 0-17990, partition values: [empty row]
14:02:55.694 [Executor task launch worker for task 0.0 in stage 34.0 (TID 68)] INFO  org.apache.parquet.filter2.compat.FilterCompat - Filtering using predicate: noteq(IATA_CODE, null)
14:02:55.718 [Executor task launch worker for task 0.0 in stage 34.0 (TID 68)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 34.0 (TID 68). 3656 bytes result sent to driver
14:02:55.719 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 34.0 (TID 68) in 36 ms on 172.23.57.81 (executor driver) (1/1)
14:02:55.719 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed Task

14:02:56.548 [Thread-3] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 30.979602 ms
14:02:56.559 [Thread-3] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
14:02:56.561 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 24 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
14:02:56.561 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 37 (parquet at NativeMethodAccessorImpl.java:0)
14:02:56.561 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List(ShuffleMapStage 36)
14:02:56.561 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: List()
14:02:56.562 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting ResultStage 37 (MapPartitionsRDD[84] at parquet at NativeMethodAccessorImpl.